In [1]:
import typing

import pandas as pd

import config
import src
import requests
import tqdm
import json
import numpy as np
import logging

#import cltrier_lib as lib

import yaml
pd.set_option('display.max_colwidth', 100) 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\sstolwi\Github\llmdiv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\sstolwi\Git

In [2]:
CFG = config.Config()

In [3]:
#load data
reddit3000: pd.DataFrame = pd.read_parquet(f'{CFG.report_dir}/reddit3000.parquet')
reddit3000.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI
0,t3_1bx6qw,,100movies365days,DTX120: #87 - Nashville,none,announcement,True,NaN,None,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."
1,t1_c9b2nyd,I've wanted to watch this for a long time. I was also turned off by the country western aspect.,100movies365days,DTX120: #87 - Nashville,t3_1bx6qw,elaboration,None,1.0,mcgrewf10,"[""The user has wanted to watch something for a long time"", ""The country western aspect of someth...","[0.4254729151725769, -0.01032189279794693, -0.6786631941795349, 0.32541653513908386, 0.298534691..."
2,t1_c9b30i1,You strike me as the type who would appreciate it. I would give it a go. This is also my first A...,100movies365days,DTX120: #87 - Nashville,t1_c9b2nyd,elaboration,None,2.0,None,"[""Magnolia has a similar structure to Altman's films with no lead character"", ""Boogie Nights and...","[0.7148110270500183, -0.15317122638225555, -0.5416110157966614, -0.1810915321111679, -0.36302334..."
3,t1_c9b6sj0,"Yeah, I've always heard that Altman was famous for his ensemble casts. But I, too, have never se...",100movies365days,DTX120: #87 - Nashville,t1_c9b30i1,elaboration,None,3.0,mcgrewf10,"[""Altman is famous for his ensemble casts"", ""The user has never seen an Altman film""]","[0.7345885634422302, -0.177871972322464, -0.609611451625824, -0.0007055625319480896, -0.08616860..."
4,t3_omv7p,"Alright guys, little background about myself. I'm a good looking, 23 year old male, have had mod...",100sets,"Male, 23 years old. Going for 100 sets!",none,announcement,True,NaN,Keatonus,"[""The writer has had modest success with women in the past"", ""The writer lacks killer instinct a...","[0.5479655265808105, -0.197927325963974, -0.4277328550815582, 0.18708674609661102, 0.06689110398..."


In [6]:
#select the rows in the dataset that have the a showName with more than cutoff comments:   
cutoff = 40
selectedposts = reddit3000[reddit3000['thread_title'].isin(reddit3000['thread_title'].value_counts()[reddit3000['thread_title'].value_counts() >= cutoff].index)]

In [7]:
selectedposts.head()

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI
1175,t3_17feeu,"I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appre...",android,What ROM are you currently running and what do you like/dislike about it?,none,question,True,NaN,RipperDaVe,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."
1176,t1_c84zkny,ParanoidKANGdroid. Has AOKP paranoid android and CM10 combined. Absolutely genius. And it is sta...,android,What ROM are you currently running and what do you like/dislike about it?,t3_17feeu,answer,None,1.0,None,"[""AOKP paranoid android and CM10 can be combined"", ""The combination of AOKP paranoid android and...","[0.6152559518814087, -0.22806480526924133, -0.7964597344398499, 0.3017873466014862, 0.0959199368..."
1177,t1_c84znti,Isn't this kind of like PAC (Paranoid/AOKP/CM)?,android,What ROM are you currently running and what do you like/dislike about it?,t1_c84zkny,question,None,2.0,OmegaVesko,"[""This is similar to PAC (Paranoid/AOKP/CM).""]","[0.4787346422672272, -0.1805042177438736, -0.42209142446517944, 0.4346879720687866, 0.3241922259..."
1178,t1_c850f55,"Omg. That's what it should be called, way cooler than paranoidKANGdroid.",android,What ROM are you currently running and what do you like/dislike about it?,t1_c84znti,appreciation,None,3.0,None,"[""ParanoidKANGdroid is not as cool as the suggested name.""]","[0.35291609168052673, -0.2874097526073456, -0.6045865416526794, 0.23958219587802887, 0.192347049..."
1179,t1_c850if1,http://forum.xda-developers.com/showthread.php?t=2015607,android,What ROM are you currently running and what do you like/dislike about it?,t1_c850f55,elaboration,None,4.0,OmegaVesko,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."


In [8]:
selectedposts['thread_title'].value_counts()

thread_title
What ROM are you currently running and what do you like/dislike about it?          41
Get rid of unwanted system apps (adb shell pm hide)                                41
If your country has a 30-35 hour work week, what do you do with your free time?    41
Name: count, dtype: int64

In [7]:
#try new Achim approach of first expanding the meaning of the post based on context and then classifying the addition of new information per post:


In [24]:
SYSTEM_expansion: str = \
    """
        Instruction:\nYou verbalize the meaning of a social media post given the preceding comments in the thread. You will be provided with a thread of comments enclosed in chevrons  <..> and a target post enclosed in double chevrons <<...>>.\nYou respond with the target reply-post, but, if it contains words alluding to information in the preceding post, you expand on these words so that their meaning is preserved without the preceding comments. Your response is worded as a social media post. If the meaning of the target reply-post is not affected by the context of the preceding posts, you reply with a verbatim copy of the target reply-post.\\n\\nText:"
	"""

In [64]:
SYSTEM_expansion2: str = \
    """
# Instruction\n\nRephrase a social media post to reflect its meaning within the context of a conversation thread:\n\n1. You'll receive a thread in chevrons `<...>` and a target post in double chevrons `<<...>>`.\n2. If the post is clear without context, repeat it as-is.\n3. If the post depends on context, expand it to include necessary details.\n4. Respond with only the expanded post. \n5. If the post does not refer to context provided in the thread, or if in doubt, respond with the exactly repeating the target post as you received it. If no target post was presented reply with an empty list [] \n\n## Example\n\n**Input**:\n- Thread: <'Comment 1', 'Comment 2', 'Comment 3'>\n- Target reply: `<<This is so relatable!>>`\n\n**Output**:\n- '[context from previous comments] is so relatable!'\n\n**Text:
    """

In [5]:
MODEL: str = 'llama3.1:8b-instruct-q6_K' # options: 'gemma:7b-instruct-q6_K', 'gemma2:27b-instruct-q6_K', 'llama3.1:8b-instruct-q6_K', 'llama3.1:70b-instruct-q6_K', 'mistral:7b-instruct-v0.3-q6_K', 'mistral-large:123b-instruct-2407-q6_K', 'mixtral:8x7b-instruct-v0.1-q6_K', 'mixtral:8x22b-instruct-v0.1-q6_K', 'phi3:14b-medium-128k-instruct-q6_K' or 'qwen2:72b-instruct-q6_K'


In [11]:
options_zero = """
seed: 42
temperature: 0
"""

options_zero = yaml.safe_load(options_zero)

In [65]:
requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODEL,
                                'system': SYSTEM_expansion2,
                                'prompt': f'"Thread":\n<{selectedposts["body"][:5].to_list()}>, "Target reply":<<{selectedposts["body"][5:6].to_list()}>>',
                                'options': options_zero
                                }).json()

{'id': '58c3719a-9ab5-11ef-b791-b5b8664f6fd3',
 'timestamp': '2024-11-04T14:01:54.073846',
 'model': 'llama3.1:8b-instruct-q6_K',
 'prompt': [{'role': 'system',
   'content': "\n# Instruction\n\nRephrase a social media post to reflect its meaning within the context of a conversation thread:\n\n1. You'll receive a thread in chevrons `<...>` and a target post in double chevrons `<<...>>`.\n2. If the post is clear without context, repeat it as-is.\n3. If the post depends on context, expand it to include necessary details.\n4. Respond with only the expanded post. \n5. If the post does not refer to context provided in the thread, or if in doubt, respond with the exactly repeating the target post as you received it. If no target post was presented reply with an empty list [] \n\n## Example\n\n**Input**:\n- Thread: <'Comment 1', 'Comment 2', 'Comment 3'>\n- Target reply: `<<This is so relatable!>>`\n\n**Output**:\n- '[context from previous comments] is so relatable!'\n\n**Text:\n    "},
  {'r

In [22]:
selectedposts["body"][:5].to_list()

["I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appreciated, thanks!\n\nEdit: Happy to hear ideas from other phones as well, thanks for the input",
 'ParanoidKANGdroid. Has AOKP paranoid android and CM10 combined. Absolutely genius. And it is stable! The customization is incredible. On jellybean too.\n\n**EDIT:** link for OP, on the I747 GS3 \n\nhttp://forum.xda-developers.com/showthread.php?t=1944523',
 "Isn't this kind of like PAC (Paranoid/AOKP/CM)?",
 "Omg. That's what it should be called, way cooler than paranoidKANGdroid.",
 'http://forum.xda-developers.com/showthread.php?t=2015607']

In [21]:
selectedposts["body"][5:6].to_list()

['Thanks this looks really good']

In [13]:
selectedposts['preceding_index'] = selectedposts.index.to_series().shift(1)

C:\Users\sstolwi\AppData\Local\Temp\ipykernel_26532\1022245724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selectedposts['preceding_index'] = selectedposts.index.to_series().shift(1)


In [14]:
selectedposts['preceding_index'].isna().sum()

np.int64(1)

In [17]:
selectedposts.drop(columns=['preceding_index'], inplace=True)

C:\Users\sstolwi\AppData\Local\Temp\ipykernel_26532\714106042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selectedposts.drop(columns=['preceding_index'], inplace=True)


In [79]:
selectedposts.head(10)

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI
1175,t3_17feeu,"I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appre...",android,What ROM are you currently running and what do you like/dislike about it?,none,question,True,NaN,RipperDaVe,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."
1176,t1_c84zkny,ParanoidKANGdroid. Has AOKP paranoid android and CM10 combined. Absolutely genius. And it is sta...,android,What ROM are you currently running and what do you like/dislike about it?,t3_17feeu,answer,None,1.0,None,"[""AOKP paranoid android and CM10 can be combined"", ""The combination of AOKP paranoid android and...","[0.6152559518814087, -0.22806480526924133, -0.7964597344398499, 0.3017873466014862, 0.0959199368..."
1177,t1_c84znti,Isn't this kind of like PAC (Paranoid/AOKP/CM)?,android,What ROM are you currently running and what do you like/dislike about it?,t1_c84zkny,question,None,2.0,OmegaVesko,"[""This is similar to PAC (Paranoid/AOKP/CM).""]","[0.4787346422672272, -0.1805042177438736, -0.42209142446517944, 0.4346879720687866, 0.3241922259..."
1178,t1_c850f55,"Omg. That's what it should be called, way cooler than paranoidKANGdroid.",android,What ROM are you currently running and what do you like/dislike about it?,t1_c84znti,appreciation,None,3.0,None,"[""ParanoidKANGdroid is not as cool as the suggested name.""]","[0.35291609168052673, -0.2874097526073456, -0.6045865416526794, 0.23958219587802887, 0.192347049..."
1179,t1_c850if1,http://forum.xda-developers.com/showthread.php?t=2015607,android,What ROM are you currently running and what do you like/dislike about it?,t1_c850f55,elaboration,None,4.0,OmegaVesko,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."
1180,t1_c854ao0,Thanks this looks really good,android,What ROM are you currently running and what do you like/dislike about it?,t1_c84zkny,appreciation,None,2.0,RipperDaVe,"[""This looks really good""]","[0.34148475527763367, -0.018923237919807434, -0.6320880651473999, 0.2847198247909546, 0.26829630..."
1181,t1_c854h9b,None,android,What ROM are you currently running and what do you like/dislike about it?,t1_c854ao0,elaboration,None,3.0,None,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."
1182,t1_c854nw3,None,android,What ROM are you currently running and what do you like/dislike about it?,t1_c854h9b,appreciation,None,4.0,None,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."
1183,t1_c854v6p,None,android,What ROM are you currently running and what do you like/dislike about it?,t1_c854ao0,None,None,3.0,None,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713..."
1184,t1_c84zrl0,"This won't help you at all, but I jumped into this thread because I didn't realize you were talk...",android,What ROM are you currently running and what do you like/dislike about it?,t3_17feeu,None,None,1.0,autovonbismarck,"[""Hephappy's CM10 is super stable and everything works great"", ""There isn't enough support for 4...","[0.546505331993103, -0.19667872786521912, -0.7661574482917786, 0.044912151992321014, -0.11970678..."


In [56]:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = selectedposts[:3].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            print(row['body'])
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            print('threadset=' + str(threadset))
            print('index=' + str(index))
            print(f'prompt: "Thread":\n<{df["body"][:threadset].to_list()}>, "Target reply":<<{df["body"][index]}>>')

100%|██████████| 1/1 [00:00<00:00, 106.67it/s]

I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appreciated, thanks!

Edit: Happy to hear ideas from other phones as well, thanks for the input
threadset=1175
index=1176
prompt: "Thread":
<["I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appreciated, thanks!\n\nEdit: Happy to hear ideas from other phones as well, thanks for the input", 'ParanoidKANGdroid. Has AOKP paranoid android and CM10 combined. Absolutely genius. And it is stable! The customization is incredible. On jellybean too.\n\n**EDIT:** link for OP, on the I747 GS3 \n\nhttp://forum.xda-developers.com/showthread.php?t=1944523', "Isn't this kind of like PAC (Paranoid/AOKP/CM)?"]>, "Target reply":<<ParanoidKANGdroid. Has AOKP paranoid android and CM10 combined. Absolutely genius. And it is stable! The customization is incredible. On jellybean too.

**EDIT:** link for OP, on the I747 GS3 

http://forum.xda-developers.com/showthread.php?t=1944

In [55]:
df["body"][1177]


"Isn't this kind of like PAC (Paranoid/AOKP/CM)?"

In [66]:
#test low temperature claims per comment:
chunked_result: typing.List[pd.DataFrame] = []
GROUPER='thread_title'
groupeddata = selectedposts[:10].groupby(GROUPER)
for group, df in tqdm.tqdm(groupeddata):
    df['preceding_index'] = df.index.to_series().shift(1)
    for index, row in df.iterrows():
        if pd.isna(row['preceding_index']):
            chunked_result.append(
                pd.DataFrame(
                    data=[row['body']],
                    columns=['post_expansion']
                )
            )
        if pd.notna(row['preceding_index']):
            threadset = int(row['preceding_index'])
            try: 
                chunked_result.append(
                    pd.DataFrame(
                        data=[
                            requests.post(
                                'https://inf.cl.uni-trier.de/',
                                json={
                                    'model': MODEL,
                                    'system': SYSTEM_expansion2,
                                    'prompt': f'"Thread":\n<{df["body"][:threadset].to_list()}>, "Target reply":<<{df["body"][index]}>>',
                                    'options': options_zero
                                    }).json()['response']                       
                        ],
                        columns=['post_expansion']
                    )
                )
            except json.JSONDecodeError:
                print("invalid json response, skipping to next batch")
    

100%|██████████| 1/1 [00:08<00:00,  8.95s/it]


In [67]:
chunked_result

[                                                                                        post_expansion
 0  I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appre...,
                                                                                         post_expansion
 0  The user has found a ROM combination that they highly recommend: AOKP paranoid android and CM10 ...,
                                                                                         post_expansion
 0  The combination of Paranoid Android and AOKP with CM10 is similar to the concept of PAC, which c...,
                                                                                         post_expansion
 0  The person who suggested calling the combination of Paranoid Android, AOKP, and CM10 "PAC" think...,
                                                                                         post_expansion
 0  This won't help you at all, but I jumped into this threa

In [74]:
expanded_posts = pd.concat(chunked_result, ignore_index=True)
print(expanded_posts)

                                                                                        post_expansion
0  I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appre...
1  The user has found a ROM combination that they highly recommend: AOKP paranoid android and CM10 ...
2  The combination of Paranoid Android and AOKP with CM10 is similar to the concept of PAC, which c...
3  The person who suggested calling the combination of Paranoid Android, AOKP, and CM10 "PAC" think...
4  This won't help you at all, but I jumped into this thread because I didn't realize you were talk...
5  The ROM you're considering flashing seems promising, as ParanoidKANGdroid has combined AOKP para...
6                                                                                                   []
7                                                                                                   []
8                                                                        

In [81]:
expanded_posts.post_expansion.to_list()

["I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appreciated, thanks!\n\nEdit: Happy to hear ideas from other phones as well, thanks for the input",
 "The user has found a ROM combination that they highly recommend: AOKP paranoid android and CM10 combined. They consider it to be absolutely genius, stable, and incredibly customizable. It's even available on Jellybean for the I747 GS3 phone.",
 'The combination of Paranoid Android and AOKP with CM10 is similar to the concept of PAC, which combines Paranoid, AOKP, and CM.',
 'The person who suggested calling the combination of Paranoid Android, AOKP, and CM10 "PAC" thinks that name is a great idea, and even better than the original suggestion of "paranoidKANGdroid".',
 'This won\'t help you at all, but I jumped into this thread because I didn\'t realize you were talking about a specific phone.\n\nFor anybody who has an **LG Optimus One (P500)** try hephappy\'s CM10. he his no longer developing (

In [83]:
selectedpostshead.index

Index([1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184], dtype='int64')

In [84]:
expanded_posts.index

RangeIndex(start=0, stop=10, step=1)

In [89]:
selectedpostshead = selectedposts.head(10)
expanded_posts.index = selectedpostshead.index
selectedpostshead = selectedpostshead.join(expanded_posts)

In [91]:
selectedpostshead.columns

Index(['id', 'body', 'subreddit', 'thread_title', 'majority_link',
       'majority_type', 'is_first_post', 'post_depth', 'author', 'claim_oplow',
       'claim_optlow_MXBAI', 'post_expansion'],
      dtype='object')

In [86]:
SYSTEM_claim: str = \
    """
        Instruction:

        You are a text annotation assistant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.
        Claims are defined as the main assertion or conclusion of an argument.
        You summarize each claim into a short simple sentence.

        Response format:

        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].

        Response format template:
        
        ["claim 1", "claim 2", ... "claim x"]
	"""

In [94]:
selectedpostshead['post_expansion'].iloc[2]

'The combination of Paranoid Android and AOKP with CM10 is similar to the concept of PAC, which combines Paranoid, AOKP, and CM.'

In [96]:
requests.post(
                            'https://inf.cl.uni-trier.de/',
                            json={
                                'model': MODEL,
                                'system': SYSTEM_claim,
                                'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                        + f'Check whether your answer only consists of a list of claims. \n"Posts":\n<{selectedpostshead['post_expansion'].iloc[2]}>',
                                'options': options_zero
                                }).json()

{'id': '5f3edfe4-9ab7-11ef-b791-b5b8664f6fd3',
 'timestamp': '2024-11-04T14:16:23.942464',
 'model': 'llama3.1:8b-instruct-q6_K',
 'prompt': [{'role': 'system',
   'content': '\n        Instruction:\n\n        You are a text annotation assistant. Analyze a collection of social media comments enclosed in chevrons <..>. Identify and list the claims within these comments. Claims can be related to events, issues, opinions or concerns in relation to the specified topic.\n        Claims are defined as the main assertion or conclusion of an argument.\n        You summarize each claim into a short simple sentence.\n\n        Response format:\n\n        You provide only the list of claims, separated by commas, without any additional text or explanations. If no claims can be identified, return an empty list [].\n\n        Response format template:\n        \n        ["claim 1", "claim 2", ... "claim x"]\n\t'},
  {'role': 'user',
   'content': 'The following set of social media posts are replies 

In [97]:
#test low temperature claims per comment:
chunked_result: typing.List[pd.DataFrame] = []
for index, row in tqdm.tqdm(selectedpostshead.iterrows()):
    try: 
        chunked_result.append(
            pd.DataFrame(
                data=[
                    requests.post(
                        'https://inf.cl.uni-trier.de/',
                        json={
                            'model': MODEL,
                            'system': SYSTEM_claim,
                            'prompt': f'The following set of social media posts are replies to a news- or infotainment-post. '
                                    + f'Check whether your answer strictly adheres to the specified format. \n"Posts":\n<{row["post_expansion"]}>',
                            'options': options_zero
                            }).json()['response']                       
                ],
                columns=['claims']
            )
        )
        
    except json.JSONDecodeError:
        print("invalid json response, skipping to next batch")

10it [00:05,  1.71it/s]


In [98]:
claims = pd.concat(chunked_result, ignore_index=True)
print(claims)

                                                                                                claims
0                                                                                                   []
1  ["AOKP paranoid android and CM10 is a recommended ROM combination", "It is absolutely genius", "...
2         ["The combination of Paranoid Android and AOKP with CM10 is similar to the concept of PAC."]
3                                                                                                   []
4  ["The LG Optimus One (P500) can run CM10", "CM10 is super stable for the LG Optimus One (P500)",...
5  ["The ROM being considered is promising", "It combines AOKP paranoid android and CM10 with great...
6                                                                                                   []
7                                                                                                   []
8                                                                        

In [101]:
claims.index = selectedpostshead.index
selectedpostshead.loc[:, "claims"] = claims

In [102]:
selectedpostshead

,id,body,subreddit,thread_title,majority_link,majority_type,is_first_post,post_depth,author,claim_oplow,claim_optlow_MXBAI,post_expansion,claims
1175,t3_17feeu,"I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appre...",android,What ROM are you currently running and what do you like/dislike about it?,none,question,True,NaN,RipperDaVe,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713...","I'm rooted on an S3 and debating what ROM I'd like to flash, any and all opinions would be appre...",[]
1176,t1_c84zkny,ParanoidKANGdroid. Has AOKP paranoid android and CM10 combined. Absolutely genius. And it is sta...,android,What ROM are you currently running and what do you like/dislike about it?,t3_17feeu,answer,None,1.0,None,"[""AOKP paranoid android and CM10 can be combined"", ""The combination of AOKP paranoid android and...","[0.6152559518814087, -0.22806480526924133, -0.7964597344398499, 0.3017873466014862, 0.0959199368...",The user has found a ROM combination that they highly recommend: AOKP paranoid android and CM10 ...,"[""AOKP paranoid android and CM10 is a recommended ROM combination"", ""It is absolutely genius"", ""..."
1177,t1_c84znti,Isn't this kind of like PAC (Paranoid/AOKP/CM)?,android,What ROM are you currently running and what do you like/dislike about it?,t1_c84zkny,question,None,2.0,OmegaVesko,"[""This is similar to PAC (Paranoid/AOKP/CM).""]","[0.4787346422672272, -0.1805042177438736, -0.42209142446517944, 0.4346879720687866, 0.3241922259...","The combination of Paranoid Android and AOKP with CM10 is similar to the concept of PAC, which c...","[""The combination of Paranoid Android and AOKP with CM10 is similar to the concept of PAC.""]"
1178,t1_c850f55,"Omg. That's what it should be called, way cooler than paranoidKANGdroid.",android,What ROM are you currently running and what do you like/dislike about it?,t1_c84znti,appreciation,None,3.0,None,"[""ParanoidKANGdroid is not as cool as the suggested name.""]","[0.35291609168052673, -0.2874097526073456, -0.6045865416526794, 0.23958219587802887, 0.192347049...","The person who suggested calling the combination of Paranoid Android, AOKP, and CM10 ""PAC"" think...",[]
1179,t1_c850if1,http://forum.xda-developers.com/showthread.php?t=2015607,android,What ROM are you currently running and what do you like/dislike about it?,t1_c850f55,elaboration,None,4.0,OmegaVesko,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713...","This won't help you at all, but I jumped into this thread because I didn't realize you were talk...","[""The LG Optimus One (P500) can run CM10"", ""CM10 is super stable for the LG Optimus One (P500)"",..."
1180,t1_c854ao0,Thanks this looks really good,android,What ROM are you currently running and what do you like/dislike about it?,t1_c84zkny,appreciation,None,2.0,RipperDaVe,"[""This looks really good""]","[0.34148475527763367, -0.018923237919807434, -0.6320880651473999, 0.2847198247909546, 0.26829630...","The ROM you're considering flashing seems promising, as ParanoidKANGdroid has combined AOKP para...","[""The ROM being considered is promising"", ""It combines AOKP paranoid android and CM10 with great..."
1181,t1_c854h9b,None,android,What ROM are you currently running and what do you like/dislike about it?,t1_c854ao0,elaboration,None,3.0,None,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713...",[],[]
1182,t1_c854nw3,None,android,What ROM are you currently running and what do you like/dislike about it?,t1_c854h9b,appreciation,None,4.0,None,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045, 0.351623713...",[],[]
1183,t1_c854v6p,None,android,What ROM are you currently running and what do you like/dislike about it?,t1_c854ao0,None,None,3.0,None,[],"[0.3085225224494934, 0.0025453902781009674, -0.632075309753418, 0.29504382610321045,